In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from matplotlib import rc
rc("font", family = "serif",size=20)
rc("figure",figsize=(9,6))
rc("figure",facecolor="white")
%config InlineBackend.figure_format = 'retina'

# %load_ext autoreload
# %autoreload 2
import skimage.io as io
import re
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage import io
from skimage.color import rgb2grey

In [2]:
l=  os.listdir('images/')

# l.pop(99)
s = l[0]
#re.search('(?<=_)\d+',s)
# re.search()


In [3]:
a = np.sort(np.array([(int(i[2:10]),float(i[13:-4]),"images/"+i) for i in l ],dtype=[("time",np.int),("z",np.float),("name","U40")]))
ind = np.argsort(a)

# times = np.unique(a[:,0])
# arr = a[a[:,0]==times[0]]
a[2]

(86530125, -0.3, 'images/6_86530125_z_-0.3.jpg')

In [4]:
def imread_convert(f):
    return rgb2grey(io.imread(f))

In [5]:
a = np.sort(np.array([(int(i[2:10]),float(i[13:-4]),"images/"+i,imread_convert(f"images/{i}")) for i in l ],dtype=[("time",np.int),("z",np.float),("name","U40"),("image","(450,540)float")]))
ind = np.argsort(a)

# times = np.unique(a[:,0])
# arr = a[a[:,0]==times[0]]


In [6]:
a.dtype.names[:-1]

('time', 'z', 'name')

In [7]:
column = 'z'
remaining_identifiers = [i for i in dtype.keys()  if i not in ['name','image']]
np.unique(a[remaining_identifiers])

NameError: name 'dtype' is not defined

In [ ]:
remaining_identifiers

In [ ]:
l=[0,3,1,10]

In [ ]:
dtype = dict(a.dtype.fields) #make a shallow copy
# dtype.pop('z')
dtype.pop('name')
dtype.keys()

In [9]:
def reduce(arr,column,function=np.mean):
    #calc new dtype
    dtype = dict(a.dtype.fields) #make a shallow copy
    dtype.pop(column)
    dtype.pop('name')
    
    #which identifying columsn will survive
    remaining_identifiers = [i for i in dtype.keys()  if i not in [column,'name','image']]
    
    #get number of unique entries in output
    unique = np.unique(arr[remaining_identifiers])
    # i think that this will preserve order??
    
    col = arr[column]
    #need unique remaining indentify columns
    
    
    out = np.zeros(len(unique),dtype= dtype)
    print(dtype)
    for i,v in enumerate(unique):
        ind = a[remaining_identifiers]==v
        #get indices ind = col == i
        im = function(arr['image'][ind],axis=0)

        out[i] = (*v,im)
    return out
out = reduce(a,'z')

{'time': (dtype('int64'), 0), 'image': (dtype(('<f8', (450, 540))), 176)}


In [ ]:
play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=len(out)-1,
    step=1,
#     description="Press play",
    disabled=False,
    loop=True
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))

fig = plt.figure()
im = plt.imshow(out[0]['image'])

def update(i):
    im.set_data(out[i]['image'])
    fig.canvas.draw()
    
interact(update,i=slider)
widgets.HBox([play, slider])


In [ ]:
plt.close(fig)

In [12]:
import ipywidgets as widgets
fig = plt.figure()
im = plt.imshow(out[0]['image'])
def on_value_change(change):
    i = change['new']
    im.set_data(out[i]['image'])
    fig.canvas.draw()

slider = widgets.IntSlider(min=0, max=len(out)-1, step=1, continuous_update=True)
play = widgets.Play(min=0, interval=200)

slider.observe(on_value_change, 'value')
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play,slider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
times = np.sort(np.unique(a['time']))
times

array([86530125, 86787171, 87328484, 87717515, 88583890, 89446265,
       90311234, 91172859, 92037140, 92897218, 93757406, 94619765])

In [45]:
t0 = a[a['time']==times[0]]
t1 = a[a['time']==times[-1]]
int_imshow_interact(t0,title_col='z')
int_imshow_interact(t1,title_col='z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=10), Output()), _dom_classes=('widget-interact',…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=30), Output()), _dom_classes=('widget-interact',…

In [44]:
pwd

'/home/ian/Documents/hekstra-lab/six'

# finding all in focus images

Find a good in focus image at the first time step. then at each subsequent time step take the image that has the highest ssim to the previous time step best image.

In [38]:
from skimage.measure import compare_ssim as ssim
#ssim(t0['image'][4],t1['image'][i] 
np.argmax([ssim(t0['image'][3],t1['image'][i]) for i in range(len(t1))])

21

In [41]:
def int_imshow_interact(arr,title_col=None):
#     play = widget.Play()
    i_widget = widgets.IntSlider(min=0,max=len(arr)-1,value=0)
    fig = plt.figure()
    ax = plt.gca()
    im = plt.imshow(arr[0]['image'])
    
    def update(i):
        if title_col is not None:
            ax.set_title(f"{title_col} = {arr[i][title_col]}")
        im.set_data(arr[i]['image'])
        fig.canvas.draw()
    interact(update,i=i_widget)
int_imshow_interact(out,title_col='time')
# plt.close()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=11), Output()), _dom_classes=('widget-interact',…

In [ ]:
fig = plt.figure()
plt.imshow(out[0]['image'])
plt.show()

In [ ]:
plt.figure()
plt.imshow(out[2]['image'])
plt.show()

In [ ]:
np.unique(np.array([[0,0],[0,0],[0,1],[1,0],]),axis=0)

In [ ]:
out['time']

In [ ]:
yikes = np.zeros(3,dtype=[("time",np.int),("z",np.float),("image","(450,540)float")])
yikes[0]['image'].shape

In [ ]:
from skimage.color import rgb2grey
def imread_convert(f):
    return rgb2grey(io.imread(f))
ic  = io.ImageCollection(files,load_func = imread_convert)
ic[0]

In [ ]:
fig = plt.figure()
ax = plt.gca()
im = plt.imshow(ic[0])
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
t_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);


@interact(t=widgets.IntSlider(min=0,max=len(ic)-1,step=1,value=10),z=widgets.IntSlider(min=0,max=len(ic)-1,step=1,value=10))
def update(t,z):
    ax.set_title(a[files_ind]['z'][z])
    im.set_data(ic[z])
#     print(ic[i])
    fig.canvas.draw()
   # return im
            
interact(update,t=t_widget,z=z_widget)


In [ ]:
im.

In [ ]:
%matplotlib inline
from ipywidgets import interact, widgets
from IPython.display import display
import matplotlib.pyplot as plt
import warnings

from skimage.morphology import disk
from skimage.filters import rank
from skimage.color import rgb2gray
import skimage.data


def f(Median_Size):
    selem = disk(int(Median_Size))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        img_median = rank.median(img_gray, selem=selem) 

    ax_neu.imshow(img_median, cmap="gray")
    fig.canvas.draw()
    display(fig)

image = skimage.data.camera() #plt.imread("Test.png")       
img_gray = rgb2gray(image)

fig = plt.figure(figsize=(6, 4))
ax_orig = fig.add_subplot(121) 
ax_neu = fig.add_subplot(122) 

ax_orig.imshow(img_gray, cmap="gray")
ax_neu.imshow(img_gray, cmap="gray")

In [ ]:
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

In [ ]:
interact(f, Median_Size=widgets.IntSlider(min=1,max=21,step=2,value=1));

In [ ]:
for i in range(len(ic)):
    plt.imshow(ic[i])
    plt.show()